In [ ]:
import pickle 
import pandas as pd
import torchsummary


with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\df_final.pkl', 'rb') as f:
    df = pickle.load(f)

In [2]:
df = df.dropna()
df.head()

,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,300min_MA,900min_MA,7_ema,25_ema,...,log_99_ema,log_300_ema,log_900_ema,MACD,Signal_Line,obv,rsi,vma,vol_pct,MACD-SIGNAL
time,,,,,,,,,,,,,,,,,,,,,
2022-11-14 20:17:00,0.047696,0.001045,3.640008,0.000262,-0.000404,-0.000231,-0.000075,0.00002,0.000513,-2.819888e-05,...,-0.000190,-0.000101,-0.000052,-32.190122,-42.986946,-0.023471,0.427829,-0.003950,-1.147692,10.796824
2022-11-14 20:18:00,0.751147,0.000294,-0.718221,0.000422,-0.000253,-0.000242,-0.000069,0.00002,0.000459,-3.393075e-06,...,-0.000180,-0.000098,-0.000051,-27.518817,-39.893321,-0.009338,0.437425,0.011163,14.748759,12.374503
2022-11-14 20:19:00,-0.154670,0.000703,1.388568,0.000675,-0.000139,-0.000222,-0.000065,0.00002,0.000520,5.096603e-05,...,-0.000160,-0.000093,-0.000050,-22.633165,-36.441289,0.002610,0.524746,0.003284,-1.205912,13.808125
2022-11-14 20:20:00,-0.342243,-0.000220,-1.312561,0.000625,0.000014,-0.000220,-0.000063,0.00002,0.000335,3.013098e-05,...,-0.000162,-0.000094,-0.000050,-18.832632,-32.919558,0.010468,0.623181,-0.027872,1.212726,14.086926
2022-11-14 20:21:00,0.387552,-0.000360,0.636838,0.000498,0.000163,-0.000224,-0.000061,0.00002,0.000161,1.387157e-07,...,-0.000166,-0.000096,-0.000051,-16.107056,-29.557057,-0.000436,0.639589,-0.003525,-2.132388,13.450002


In [3]:
threshold = 0.00005
target = pd.cut(df['mid_price'], bins=[-float('inf'), -threshold, threshold, float('inf')], labels=[0, 1, 2])
target.value_counts()

mid_price
2    210659
0    209904
1    103552
Name: count, dtype: int64

In [4]:
import numpy as np
from tqdm import tqdm

feature = df

# 주어진 데이터프레임
sequence_length = 64
n_samples = len(df) - sequence_length + 1

# Extract the data as a NumPy array
data_array = df.to_numpy()

# Create an empty array to store the input sequences
input_sequences = np.empty((n_samples, (df.shape[1]), sequence_length))

for i in tqdm(range(n_samples)):
    input_seq = data_array[i:i + sequence_length, :].T  # Transpose to have (24, 60) shape
    input_sequences[i] = input_seq

print(input_sequences.shape)

100%|██████████| 524052/524052 [00:02<00:00, 216097.42it/s]

(524052, 30, 64)


In [5]:
X = input_sequences[:-1]
y = target.shift(-1)[63:].dropna()

X.shape, y.shape

((524051, 30, 64), (524051,))

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 피처와 타겟 데이터 생성 (예시 데이터)
num_samples = 524052
num_channels = 1
num_classes = 3

feature = np.array(X[-100000:], dtype=np.float32)
target = np.array(y[-100000:], dtype=np.float32)

print(feature.shape, target.shape)

(100000, 30, 64) (100000,)


In [17]:
from classifier import Classifier 

dataset = TensorDataset(torch.tensor(feature, dtype=torch.float32), torch.tensor(target, dtype=torch.int64))
batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size)

print("model loading...")
# 모델 초기화
classifier_model = Classifier(input_channels=num_channels, num_classes=3)

print("optimizer loading...")
# 손실 함수 및 최적화기 설정
criterion = nn.CrossEntropyLoss()

learning_rate = 0.001

optimizer = optim.Adam(classifier_model.parameters(), lr=learning_rate)

model loading...
optimizer loading...


In [18]:
# 학습 루프
num_epochs = 100
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch in dataloader:
        inputs, labels = batch

        # 모델 예측
        outputs = classifier_model(inputs)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파 및 가중치 갱신
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 정확도 계산
        _, predicted_labels = torch.max(outputs, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

    # 에폭마다 평균 손실 및 정확도 출력
    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples * 100
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Training finished.")

Epoch [1/100], Loss: 1.0570, Accuracy: 41.38%
Epoch [2/100], Loss: 1.0533, Accuracy: 42.13%
Epoch [3/100], Loss: 1.0504, Accuracy: 41.94%
Epoch [4/100], Loss: 1.0481, Accuracy: 42.71%
Epoch [5/100], Loss: 1.0460, Accuracy: 43.51%
Epoch [6/100], Loss: 1.0441, Accuracy: 44.07%
Epoch [7/100], Loss: 1.0440, Accuracy: 43.97%
Epoch [8/100], Loss: 1.0431, Accuracy: 44.41%
Epoch [9/100], Loss: 1.0423, Accuracy: 44.64%
Epoch [10/100], Loss: 1.0401, Accuracy: 45.05%
Epoch [11/100], Loss: 1.0385, Accuracy: 45.57%
Epoch [12/100], Loss: 1.0364, Accuracy: 46.00%
Epoch [13/100], Loss: 1.0347, Accuracy: 46.42%
Epoch [14/100], Loss: 1.0341, Accuracy: 46.65%
Epoch [15/100], Loss: 1.0314, Accuracy: 47.09%
Epoch [16/100], Loss: 1.0285, Accuracy: 47.75%
Epoch [17/100], Loss: 1.0257, Accuracy: 48.29%
Epoch [18/100], Loss: 1.0219, Accuracy: 48.95%
Epoch [19/100], Loss: 1.0179, Accuracy: 49.61%
Epoch [20/100], Loss: 1.0155, Accuracy: 49.98%
Epoch [21/100], Loss: 1.0100, Accuracy: 51.03%
Epoch [22/100], Loss: 

KeyboardInterrupt: 

In [19]:
torch.save(classifier_model, "C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\bitcoin_scalping_bot\\version_4\\classifier.pt")

In [20]:
checkpoint= torch.load("C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\bitcoin_scalping_bot\\version_4\\classifier.pt")

In [28]:
checkpoint

Classifier(
  (conv2d_1): Conv2d(1, 8, kernel_size=(12, 2), stride=(12, 2))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_2): Conv2d(8, 16, kernel_size=(1, 17), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_3): Conv2d(16, 32, kernel_size=(1, 9), stride=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm1): LSTM(512, 64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=3, bias=True)
)